In [19]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import timeit
import time
from torch.optim import Adam
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt, random
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from scipy.stats import loguniform
from sklearn.svm import SVR
import pandas as pd
import timeit
import time
from joblib import dump, load
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt, random

In [20]:
def F_s(a_c_,a_t_,c_b_,phi_,only_mg=True, ac_over_1 = None):
    try:
        len(a_c_)
    except:
        a_c_ = [a_c_]
        a_t_ = [a_t_]
        c_b_ = [c_b_]
        phi_ = [phi_]
        
    a_c_ = np.array(a_c_)
    a_t_ = np.array(a_t_)
    c_b_ = np.array(c_b_)
    phi_ = np.array(phi_)

    # if ac_over_1 != None:
    #     data = split_ac(np.column_stack((a_c_,a_t_,c_b_,phi_)), ac_over_1)
    #     a_c_ = data[:, 0]
    #     a_t_ = data[:, 1]
    #     c_b_ = data[:, 2]
    #     phi_ = data[:, 3]
    
    try:
        F_s = np.zeros(len(a_c_))
        iters = len(a_c_)
    except:
        F_s = np.zeros(1)
        iters = 1
    for i in range(iters):
        a_c = a_c_[i]
        a_t = a_t_[i]
        c_b = c_b_[i]
        phi = phi_[i]
        fw = np.cos((np.pi/2)*(c_b)*np.sqrt(a_t))**(-0.5)
        if np.isnan(a_c):
            return 0
        if a_c <= 1:
            M1 = 1.13 - 0.09*(a_c)
            M2 = -0.54 + (0.89/(0.2+(a_c)))
            M3 = 0.5 - (1/(0.65+(a_c))) + 14*(1-(a_c))**(24)
            g = 1 + (0.1 + 0.35*(a_t)**2)*(1-np.sin(phi))**2
            fphi = ((a_c)**2*(np.cos(phi))**2+(np.sin(phi))**2)**(1/4)
    
        if a_c > 1:
            M1 = np.sqrt((a_c**(-1)))*(1+0.04*((a_c**(-1))))
            M2 = 0.2*((a_c**(-1)))**4
            M3 = -0.11*((a_c**(-1)))**4
            g = 1 + (0.1+0.35*((a_c**(-1)))*(a_t)**2)*(1-(np.sin(phi)))**2
            fphi = ((((a_c**(-1)))**2)*np.sin(phi)**(2)+np.cos(phi)**(2))**(1/4)
        
        if only_mg:
            Fs = (M1+M2*(a_t)**2+M3*(a_t)**4)*g
        else:
            Fs = (M1+M2*(a_t)**2+M3*(a_t)**4)*g*fphi*fw  
        F_s[i] = Fs

    return F_s

In [25]:
def sort(data):
    models = []
    model = np.unique(data[:,[0,1,2]], axis=0)
    
    for i in model:
        models.append(data[np.where((data[:,[0,1,2]] == i).all(axis=1))])

    return models 

def compare(model):
    rn_results = []
    # RN Equation
    for ex in model:
        rn_results.append(F_s(ex[0], ex[1], ex[2], ex[3]))
    
    # ML Models
    rbf_results_all = []
    rbf_results_sep = []
    nn_results_all = []
    nn_results_sep = []
    
    rbf_result = np.zeros((len(model),1))
    nn_result = np.zeros((len(model),1))
    model = np.delete(model, [2,4], 1)

    rbf_result[:,0] = rbf_reg_all.predict(model[:,:-1])
    rbf_results_all.append(rbf_result)
    nn_result[:,0] = np.squeeze(nn_reg_all.predict(model[:,:-1]), axis=-1)
    nn_results_all.append(nn_result)

    # a/c separated
    rbf_result = np.zeros((len(model),1))
    nn_result = np.zeros((len(model),1))
    if model[0,0] <= 1:
        rbf_result[:,0] = rbf_reg_le.predict(model[:,:-1])
        nn_result[:,0] = np.squeeze(nn_reg_le.predict(model[:,:-1]), axis=-1)

    elif model[0,0] > 1:
        rbf_result[:,0] = rbf_reg_g.predict(model[:,:-1])
        nn_result[:,0] = np.squeeze(nn_reg_g.predict(model[:,:-1]), axis=-1)

    else:
        print("Something wrong!")

    rbf_results_sep.append(rbf_result)
    nn_results_sep.append(nn_result)
        
    return rbf_results_all, rbf_results_sep, nn_results_all, nn_results_sep, rn_results



def plot(model_num):
    rbf_results_all, rbf_results_sep, nn_results_all, nn_results_sep, rn_results = compare(models[model_num])
    plt.scatter(models[model_num][:,-3], models[model_num][:,-1], label="Ground Truth")
    plt.plot(models[model_num][:,-3], nn_results_all[0][:,0], label="NN all", color='red')
    plt.plot(models[model_num][:,-3], nn_results_sep[0][:,0], label="NN separated", color='green')
    plt.plot(models[model_num][:,-3], rbf_results_all[0][:,0], label="RBF all", color='black')
    plt.plot(models[model_num][:,-3], rbf_results_sep[0][:,0], label="RBF separated", color='cyan')
    plt.plot(models[model_num][:,-3], rn_results[:], label="RN Equation", color='m')
    plt.title("a/c={}; a/t={}; c/b={}".format(models[model_num][0,0],
                                             models[model_num][0,1],
                                             models[model_num][0,2]))
    plt.xlabel("phi")
    plt.ylabel("Mg")
    plt.legend()
    plt.show()
    return

In [26]:
# Loading test dataset
df = pd.read_csv("1_RN_data.csv")
d = df.to_numpy()
models = sort(d)
len(models)

32

In [27]:
rbf_reg_all = load("models/rbf_all.joblib")
nn_reg_all = tf.keras.models.load_model('models/NN_all.h5')

rbf_reg_le = load("models/rbf_le.joblib")
nn_reg_le = tf.keras.models.load_model('models/NN_le.h5')

rbf_reg_g = load("models/rbf_g.joblib")
nn_reg_g = tf.keras.models.load_model('models/NN_g.h5')

In [28]:
interact(plot, model_num=(1,len(models)-1,1));

interactive(children=(IntSlider(value=16, description='model_num', max=31, min=1), Output()), _dom_classes=('w…